In [1]:
import os
import bs4
from bs4 import BeautifulSoup as soup
import pandas as pd
# import spacy
import re
import nltk
from nltk.tokenize import word_tokenize

In [2]:
def extract_html(input_html):
    with open(input_html) as inf:
        txt = inf.read()
    soup_out = soup(txt, "html.parser")
    print(input_html)
    return soup_out

In [3]:
def extract_firm(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.get('title')))   
    return None

In [4]:
def extract_ticker(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('a'):
        if element.get('title') != None :
            return(str(element.contents[0].strip()))   
    return None

In [5]:
def extract_time(input_soup):
    temp = input_soup.find(id='article_participants')
    try:
        for element in temp.findAll('p'):
            if (pd.notnull(element.text) and element.text!=''):
                if( re.search('call',element.contents[0],flags=re.IGNORECASE)):
                    call_title = element.contents[0].split()
                    return str(call_title[0]),str(call_title[1])
    except:
        return None,None

In [6]:
def test_analyst(input_soup):
    temp = input_soup.find(id='article_participants')
    for element in temp.findAll('strong'):
        if(re.search('analyst',element.contents[0],flags=re.IGNORECASE)):
            return(True)
    return(None)


In [7]:
def extract_analysts(input_soup):
    temp = input_soup.find(id='article_participants')
    test_string = []
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('analyst',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
            #print(element.contents[0].strip())
            #print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):
            #print('FOUND')
                    break
                if (pd.notnull(element.text) and element.text!=''):
                    test_string.append(element.contents[0].strip())    
            break
    return test_string

In [8]:
def extract_executives(input_soup):
    test_string = []
    temp = input_soup.find(id='article_participants')
    # print(soup_out)
    print_flag = False
    for element in temp.find_all('p'):
        if(re.search('executive',element.text,flags=re.IGNORECASE)):
            #print(element.text.strip(),'###')
            element = element.find_next()
            while  (1):
                element = element.find_next()
                # print(element.contents[0])
                # print(element.text.strip())
                if element.find('strong') or \
                element.find('div'):            
                #print('FOUND')
                    break
                test_string.append(element.text.strip())
        
            break
    return test_string

In [9]:
def is_analyst_present(analyst_list):
    if analyst_list: 
        return True
    else: 
        return False

In [10]:
def extract_name (input_string,delim='-'):
    if not pd.isnull(input_string):
        name_search = re.search('(^[^-]*)-', input_string, re.IGNORECASE)
        try:
            return name_search.group(1).strip()
        except:
            return None        
    else:
        return None
def extract_title (input_string,delim='-'):
    if not pd.isnull(input_string):
        title_search = re.search('^[^-]*-(.*)', input_string, re.IGNORECASE)
        try:
            return title_search.group(1).strip()
        except:
            return None           
        else:
            return None
    else:
        return None

In [11]:
def extract_content(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_content')
        test_string = []
        # print(temp)
        for element in temp.find_all('strong'):
            if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                while  (1):
                    element = element.find_next()
        #           print(element.text.strip())
                    if element.find('strong') or \
                    element.find('div'):
                        break
                    test_string.append(element.text.strip())    
        return test_string
    else:
        return None

In [12]:
def extract_qanda(input_soup,current_name):
#     current_name = 'Panna Sharma'
    if not (pd.isnull(current_name) ):
        temp = input_soup.find(id='article_qanda')
        test_string = []
        # print(temp)
        if temp.find_all('span'):
            for element in temp.find_all('span'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())    
        else:                
            for element in temp.find_all('strong'):
                if (re.search(current_name.strip().lower(),element.text,flags=re.IGNORECASE)):
                    while  (1):
                        element = element.find_next()
            #           print(element.text.strip())
                        if element.find('strong') or \
                        element.find('div') or \
                        element.find('span'):
                            break
                        test_string.append(element.text.strip())      
            
        return test_string
    else:
        return None

In [13]:
def basic_wc(input_text):
    #     print(input_text)
    print(type(input_text),'#####')
    if not ( pd.isnull(input_text)):
        decoded_txt=input_text.decode('utf-8')
        sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=decoded_txt)  
        tokens = word_tokenize(sent)
#         tokens = word_tokenize(decoded_txt)
#         tokens = [tok for tok in tokens if (not re.search('^[^a-zA-Z]*$',tok))]
        return len(tokens)
    else:
        return 0    
    

# def basic_wc(input_text):
#     try:
# #     if (input_text and not pd.isnull(input_text) and not None):
#         decoded_txt=input_text.decode('utf-8')
#         sent = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=decoded_txt)            
#         tokens = word_tokenize(sent)
# #         tokens = [tok for tok in tokens if (not re.search('^[^a-zA-Z]*$',tok))]
#         return len(tokens)
#     except:
#         return 0        

In [14]:
collection = "html"

# n_test=50
file_list = os.listdir(collection)
# for i in range(len(file_list)):
# for i in range(n_test):
#     print(file_list[i])
print(len(file_list))

91979


In [15]:
collection = "html"

n_test=2
file_list = os.listdir(collection)
out_list =[]
# for i in range(len(file_list)):
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 

for i in range(n_test):
    with open(collection + '/' + file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
        

FILE	ANALYST_Flag	FIRM	Quarter	Year


In [17]:
out_pd = pd.DataFrame(out_list)
out_pd.columns=['file']
out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
out_pd['quarter_year'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
out_pd['analyst_present'] = out_pd.apply(lambda x: test_analyst(x['text']), axis=1)
out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

s = out_pd.analysts.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('analyst')
out_analyst = out_pd.join(s)  

out_analyst['analyst_name'] = out_analyst.apply(lambda x: extract_name(x['analyst']), axis=1)
out_analyst['analyst_firm'] = out_analyst.apply(lambda x: extract_title(x['analyst']), axis=1)


out_analyst['analyst_content'] = out_analyst.apply(lambda x: \
                                         extract_content(x['text'],x['analyst_name']), axis=1)

out_analyst['analyst_qanda'] = out_analyst.apply(lambda x: \
                                         extract_qanda(x['text'],x['analyst_name']), axis=1)

# out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
#                                          basic_wc(x['analyst_qanda']), axis=1)

# out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
#                                          basic_wc(x['analyst_content']), axis=1)
# out_analyst['analyst_total_count'] = out_analyst.analyst_qna_count + out_analyst.analyst_content_count
out_analyst=out_analyst.reset_index(level=1, drop=True)
# out_analyst
s = out_analyst.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('executive')
out_analyst = out_analyst.join(s)                      

out_analyst['exec_name'] = out_analyst.apply(lambda x: extract_name(x['executive']), axis=1)
out_analyst['exec_title'] = out_analyst.apply(lambda x: extract_title(x['executive']), axis=1)


html/1.html


In [18]:
# out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
out_analyst.drop(['text','analyst_present','analysts','executive','execs','analyst'],axis=1)\
.to_csv('analyst_out_foringestion.csv',sep=',', encoding = 'utf-8')
# out_executives.select({[}'file','quarter_yer'])

In [91]:
out_analyst.dtypes

file               object
text               object
quarter_year       object
executive_firm     object
analyst_present    object
ticker             object
analysts           object
execs              object
analyst            object
analyst_name       object
analyst_firm       object
analyst_content    object
analyst_qanda      object
executive          object
exec_name          object
exec_title         object
dtype: object

In [92]:
out_analyst

,file,text,quarter_year,executive_firm,analyst_present,ticker,analysts,execs,analyst,analyst_name,analyst_firm,analyst_content,analyst_qanda,executive,exec_name,exec_title
0,1.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Actions Semiconductor Co., Ltd.",None,ACTS,[],"[Zhenyu Zhou - CEO, Nigel Liu - CFO, Elaine Ke...",NaN,None,None,None,None,Zhenyu Zhou - CEO,Zhenyu Zhou,CEO
0,1.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Actions Semiconductor Co., Ltd.",None,ACTS,[],"[Zhenyu Zhou - CEO, Nigel Liu - CFO, Elaine Ke...",NaN,None,None,None,None,Nigel Liu - CFO,Nigel Liu,CFO
0,1.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Actions Semiconductor Co., Ltd.",None,ACTS,[],"[Zhenyu Zhou - CEO, Nigel Liu - CFO, Elaine Ke...",NaN,None,None,None,None,Elaine Ketchmere - IR,Elaine Ketchmere,IR
1,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Paul Knight - Janney Montgomery Scott,Paul Knight,Janney Montgomery Scott,[],[Congratulations on the quarter. Regarding the...,Brian - IR,Brian,IR
1,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Paul Knight - Janney Montgomery Scott,Paul Knight,Janney Montgomery Scott,[],[Congratulations on the quarter. Regarding the...,Panna Sharma - President and CEO,Panna Sharma,President and CEO
1,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Paul Knight - Janney Montgomery Scott,Paul Knight,Janney Montgomery Scott,[],[Congratulations on the quarter. Regarding the...,Edward Sitar - CFO and Treasurer,Edward Sitar,CFO and Treasurer
2,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Ben Haynor - Feltl and Company,Ben Haynor,Feltl and Company,[],"[Good morning, gentlemen., Good, good. Very st...",Brian - IR,Brian,IR
2,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Ben Haynor - Feltl and Company,Ben Haynor,Feltl and Company,[],"[Good morning, gentlemen., Good, good. Very st...",Panna Sharma - President and CEO,Panna Sharma,President and CEO
2,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Ben Haynor - Feltl and Company,Ben Haynor,Feltl and Company,[],"[Good morning, gentlemen., Good, good. Very st...",Edward Sitar - CFO and Treasurer,Edward Sitar,CFO and Treasurer
3,10.html,"<div class=""content_part hid"" id=""article_par...","(Q1, 2015)","Cancer Genetics, Inc.",True,CGIX,"[Paul Knight - Janney Montgomery Scott, Ben Ha...","[Brian - IR, Panna Sharma - President and CEO,...",Sung Ji Nam - Cantor Fitzgerald,Sung Ji Nam,Cantor Fitzgerald,[],"[Hi, thanks for taking the questions. Was wond...",Brian - IR,Brian,IR


In [24]:
total_rows = len(file_list)
row_start = 200
row_end = 300
filename = 'analyst_out_foringestion_'+str(row_start)+'.csv'
print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 

for i in range(row_start,row_end):
    with open(collection + '/' + file_list[i]) as inf:
        if(re.search('html',file_list[i],flags=re.IGNORECASE)):
            out_list.append(file_list[i])
                
out_pd = pd.DataFrame(out_list)
out_pd.columns=['file']
out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
out_pd['quarter_year'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
out_pd['analyst_present'] = out_pd.apply(lambda x: test_analyst(x['text']), axis=1)
out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

s = out_pd.analysts.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('analyst')
out_analyst = out_pd.join(s)  

out_analyst['analyst_name'] = out_analyst.apply(lambda x: extract_name(x['analyst']), axis=1)
out_analyst['analyst_firm'] = out_analyst.apply(lambda x: extract_title(x['analyst']), axis=1)


out_analyst['analyst_content'] = out_analyst.apply(lambda x: \
                                         extract_content(x['text'],x['analyst_name']), axis=1)

out_analyst['analyst_qanda'] = out_analyst.apply(lambda x: \
                                         extract_qanda(x['text'],x['analyst_name']), axis=1)

# out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
#                                          basic_wc(x['analyst_qanda']), axis=1)

# out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
#                                          basic_wc(x['analyst_content']), axis=1)
# out_analyst['analyst_total_count'] = out_analyst.analyst_qna_count + out_analyst.analyst_content_count
out_analyst=out_analyst.reset_index(level=1, drop=True)
# out_analyst
s = out_analyst.execs.apply(pd.Series)\
              .stack()\
              .reset_index(level=1, drop=True)\
              .to_frame('executive')
out_analyst = out_analyst.join(s)                      

out_analyst['exec_name'] = out_analyst.apply(lambda x: extract_name(x['executive']), axis=1)
out_analyst['exec_title'] = out_analyst.apply(lambda x: extract_title(x['executive']), axis=1)
 
# out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
out_analyst.drop(['text','analyst_present','analysts','executive','execs','analyst'],axis=1)\
.to_csv(filename,sep=',', encoding = 'utf-8',header=False)
# out_executives.select({[}'file','quarter_yer'])    

FILE	ANALYST_Flag	FIRM	Quarter	Year
html/1.html
html/10.html
html/100.html
html/1000.html
html/10000.html
html/10001.html
html/10002.html
html/10003.html
html/10004.html
html/10005.html
html/10006.html
html/10007.html
html/10008.html
html/10009.html
html/1001.html
html/10010.html
html/10011.html
html/10012.html
html/10013.html
html/10014.html
html/10015.html
html/10016.html
html/10017.html
html/10018.html
html/10019.html
html/1002.html
html/10020.html
html/10021.html
html/10022.html
html/10023.html
html/10024.html
html/10025.html
html/10026.html
html/10027.html
html/10028.html
html/10029.html
html/1003.html
html/10030.html
html/10031.html
html/10032.html
html/10033.html
html/10034.html
html/10035.html
html/10036.html
html/10037.html
html/10038.html
html/10039.html
html/1004.html
html/10040.html
html/10041.html
html/10042.html
html/10043.html
html/10044.html
html/10045.html
html/10046.html
html/10047.html
html/10048.html
html/10049.html
html/1005.html
html/10050.html
html/10051.html
htm

KeyboardInterrupt: 

In [16]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_extract.csv', 'wb') as csvfile:
    logwriter = csv.writer(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_MINIMAL)
#     logwriter.writerow(fieldnames)

 
    for ii in range(2,91980):
        try:
            total_rows = len(file_list)
            row_start = ii*1
            row_end = (ii*1)+1
        #     print('from: ', row_start, ' to: ', row_end)
            filename = 'extract/analyst_out_foringestion_'+str(row_start)+'.csv'
#             print('FILE\tANALYST_Flag\tFIRM\tQuarter\tYear') 
#             print(ii,'####')
            out_list =[]

            for i in range(row_start,row_end):
                with open(collection + '/' + file_list[i]) as inf:
                    if(re.search('html',file_list[i],flags=re.IGNORECASE)):
                        out_list.append(file_list[i])

            out_pd = pd.DataFrame(out_list)
            out_pd.columns=['file']
            out_pd['text'] = out_pd.apply(lambda x: extract_html('html/'+x['file']), axis=1)
            out_pd['quarter_year'] = out_pd.apply(lambda x: extract_time(x['text']), axis=1)
            out_pd['executive_firm'] = out_pd.apply(lambda x: extract_firm(x['text']), axis=1)
            out_pd['analyst_present'] = out_pd.apply(lambda x: test_analyst(x['text']), axis=1)
            out_pd['ticker'] = out_pd.apply(lambda x: extract_ticker(x['text']), axis=1)

            out_pd['analysts'] = out_pd.apply(lambda x: extract_analysts(x['text']), axis=1)
            out_pd['execs'] = out_pd.apply(lambda x: extract_executives(x['text']), axis=1)

            s = out_pd.analysts.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('analyst')
            out_analyst = out_pd.join(s)  

            out_analyst['analyst_name'] = out_analyst.apply(lambda x: extract_name(x['analyst']), axis=1)
            out_analyst['analyst_firm'] = out_analyst.apply(lambda x: extract_title(x['analyst']), axis=1)


            out_analyst['analyst_content'] = out_analyst.apply(lambda x: \
                                                     extract_content(x['text'],x['analyst_name']), axis=1)

            out_analyst['analyst_qanda'] = out_analyst.apply(lambda x: \
                                                     extract_qanda(x['text'],x['analyst_name']), axis=1)

            # out_analyst['analyst_qna_count'] = out_analyst.apply(lambda x: \
            #                                          basic_wc(x['analyst_qanda']), axis=1)

            # out_analyst['analyst_content_count'] = out_analyst.apply(lambda x: \
            #                                          basic_wc(x['analyst_content']), axis=1)
            # out_analyst['analyst_total_count'] = out_analyst.analyst_qna_count + out_analyst.analyst_content_count
            out_analyst=out_analyst.reset_index(level=1, drop=True)
            # out_analyst
            s = out_analyst.execs.apply(pd.Series)\
                          .stack()\
                          .reset_index(level=1, drop=True)\
                          .to_frame('executive')
            out_analyst = out_analyst.join(s)                      

            out_analyst['exec_name'] = out_analyst.apply(lambda x: extract_name(x['executive']), axis=1)
            out_analyst['exec_title'] = out_analyst.apply(lambda x: extract_title(x['executive']), axis=1)

            # out_analyst.drop(['text','analyst_present','analysts','executive'],axis=1)
            out_analyst.drop(['text','analyst_present','analysts','executive','execs','analyst'],axis=1)\
            .to_csv(filename,sep=',', encoding = 'utf-8',header=False)
            # out_executives.select({[}'file','quarter_yer'])    
            logwriter.writerow([ii,out_analyst.file.head(1).item(),'success'])
        except:
            print('exception: ', ii, ' ###')
            logwriter.writerow([ii,out_analyst.file.head(1).item(),'fail'])
            
            pass

html/10.html
html/100.html
html/1000.html
html/10000.html
html/10001.html
html/10002.html
('exception: ', 7, ' ###')
html/10003.html
html/10004.html
html/10005.html
html/10006.html
html/10007.html
html/10008.html
html/10009.html
html/1001.html
html/10010.html
html/10011.html
html/10012.html
html/10013.html
html/10014.html
html/10015.html
html/10016.html
html/10017.html
html/10018.html
html/10019.html
html/1002.html
html/10020.html
html/10021.html
html/10022.html
('exception: ', 29, ' ###')
html/10023.html
html/10024.html
html/10025.html
html/10026.html
html/10027.html
html/10028.html
html/10029.html
html/1003.html
html/10030.html
html/10031.html
('exception: ', 39, ' ###')
html/10032.html
html/10033.html
html/10034.html
html/10035.html
html/10036.html
html/10037.html
html/10038.html
('exception: ', 46, ' ###')
html/10039.html
html/1004.html
('exception: ', 48, ' ###')
html/10040.html
html/10041.html
('exception: ', 50, ' ###')
html/10042.html
html/10043.html
html/10044.html
html/10045.

html/1040.html
html/10400.html
html/10402.html
html/10403.html
html/10404.html
html/10405.html
html/10406.html
html/10407.html
html/10408.html
html/10409.html
html/1041.html
html/10410.html
html/10411.html
html/10412.html
html/10413.html
html/10414.html
html/10415.html
html/10416.html
html/10417.html
html/10418.html
html/10419.html
html/1042.html
('exception: ', 465, ' ###')
html/10420.html
html/10421.html
html/10422.html
('exception: ', 468, ' ###')
html/10423.html
html/10424.html
html/10425.html
html/10426.html
html/10427.html
('exception: ', 473, ' ###')
html/10428.html
('exception: ', 474, ' ###')
html/10429.html
html/1043.html
html/10430.html
html/10431.html
html/10432.html
html/10433.html
html/10434.html
html/10435.html
html/10436.html
('exception: ', 483, ' ###')
html/10437.html
html/10438.html
html/10439.html
html/1044.html
html/10440.html
html/10441.html
('exception: ', 489, ' ###')
html/10442.html
html/10443.html
html/10444.html
html/10445.html
html/10446.html
html/10447.html

html/10800.html
html/10801.html
html/10802.html
html/10803.html
html/10804.html
('exception: ', 889, ' ###')
html/10805.html
html/10806.html
html/10807.html
html/10808.html
html/10809.html
html/1081.html
('exception: ', 895, ' ###')
html/10810.html
html/10811.html
html/10812.html
html/10813.html
html/10814.html
html/10815.html
html/10816.html
html/10817.html
('exception: ', 903, ' ###')
html/10818.html
html/10819.html
html/1082.html
html/10820.html
('exception: ', 907, ' ###')
html/10821.html
html/10822.html
('exception: ', 909, ' ###')
html/10823.html
html/10824.html
html/10825.html
html/10826.html
html/10827.html
html/10828.html
html/10829.html
html/1083.html
html/10830.html
html/10831.html
html/10832.html
html/10833.html
html/10834.html
html/10835.html
html/10836.html
html/10837.html
html/10838.html
('exception: ', 926, ' ###')
html/10839.html
html/1084.html
html/10840.html
('exception: ', 929, ' ###')
html/10841.html
html/10842.html
html/10843.html
html/10844.html
('exception: ', 9

html/11194.html
html/11195.html
html/11196.html
html/11197.html
html/11198.html
html/11199.html
html/112.html
html/1120.html
html/11200.html
html/11201.html
('exception: ', 1327, ' ###')
html/11202.html
html/11203.html
html/11204.html
html/11205.html
html/11206.html
html/11207.html
html/11208.html
html/11209.html
html/1121.html
html/11210.html
html/11211.html
html/11212.html
html/11213.html
html/11214.html
html/11215.html
('exception: ', 1342, ' ###')
html/11216.html
html/11217.html
html/11218.html
html/11219.html
html/1122.html
html/11220.html
html/11221.html
html/11222.html
html/11223.html
html/11224.html
html/11225.html
html/11226.html
html/11227.html
html/11228.html
('exception: ', 1356, ' ###')
html/11229.html
html/1123.html
html/11230.html
('exception: ', 1359, ' ###')
html/11231.html
('exception: ', 1360, ' ###')
html/11232.html
html/11233.html
html/11234.html
html/11235.html
html/11236.html
html/11237.html
html/11238.html
html/11239.html
html/1124.html
html/11240.html
html/1124

html/11594.html
html/11595.html
html/11596.html
html/11597.html
html/11598.html
html/11599.html
html/116.html
html/1160.html
html/11600.html
html/11601.html
html/11602.html
html/11603.html
html/11604.html
html/11605.html
html/11606.html
html/11607.html
html/11608.html
html/11609.html
html/1161.html
html/11610.html
html/11611.html
html/11612.html
html/11613.html
html/11614.html
html/11615.html
('exception: ', 1780, ' ###')
html/11616.html
html/11617.html
html/11618.html
html/11619.html
html/1162.html
html/11620.html
html/11621.html
html/11622.html
html/11623.html
html/11624.html
html/11625.html
html/11626.html
html/11627.html
html/11628.html
html/11629.html
html/1163.html
('exception: ', 1796, ' ###')
html/11630.html
html/11631.html
html/11632.html
html/11633.html
html/11634.html
html/11635.html
html/11636.html
('exception: ', 1803, ' ###')
html/11637.html
html/11638.html
html/11639.html
('exception: ', 1806, ' ###')
html/1164.html
html/11640.html
html/11641.html
html/11642.html
html/11

...

html/91729.html
html/9173.html
html/91730.html
('exception: ', 89905, ' ###')
html/91731.html
html/91732.html
html/91733.html
html/91734.html
html/91735.html
html/91736.html
html/91737.html
html/91738.html
html/91739.html
html/9174.html
html/91740.html
html/91741.html
html/91742.html
html/91743.html
html/91744.html
html/91745.html
html/91746.html
html/91747.html
html/91748.html
html/91749.html
html/9175.html
html/91750.html
html/91751.html
html/91752.html
html/91753.html
html/91754.html
html/91755.html
html/91756.html
html/91757.html
('exception: ', 89934, ' ###')
html/91758.html
html/91759.html
html/9176.html
html/91760.html
html/91761.html
('exception: ', 89939, ' ###')
html/91762.html
html/91763.html
html/91764.html
html/91765.html
html/91766.html
('exception: ', 89944, ' ###')
html/91767.html
html/91768.html
html/91769.html
html/9177.html
html/91770.html
html/91771.html
html/91772.html
html/91773.html
html/91774.html
('exception: ', 89953, ' ###')
html/91775.html
html/91776.html
ht

html/92120.html
('exception: ', 90339, ' ###')
html/92121.html
html/92122.html
html/92123.html
html/92124.html
html/92125.html
html/92126.html
html/92127.html
html/92128.html
('exception: ', 90347, ' ###')
html/92129.html
html/9213.html
html/92130.html
html/92131.html
html/92132.html
html/92133.html
html/92134.html
html/92135.html
html/92136.html
html/92137.html
html/92138.html
html/92139.html
html/9214.html
html/92140.html
('exception: ', 90361, ' ###')
html/92141.html
html/92142.html
html/92143.html
('exception: ', 90364, ' ###')
html/92144.html
html/92145.html
html/92146.html
html/92147.html
html/92148.html
html/92149.html
html/9215.html
html/92150.html
html/92151.html
html/92152.html
html/92153.html
html/92154.html
html/92155.html
html/92156.html
('exception: ', 90378, ' ###')
html/92157.html
html/92158.html
html/92159.html
html/9216.html
html/92160.html
html/92161.html
html/92162.html
html/92163.html
html/92164.html
('exception: ', 90387, ' ###')
html/92165.html
html/92166.html
ht

html/92505.html
html/92506.html
html/92507.html
html/92508.html
html/92509.html
html/9251.html
html/92510.html
html/92511.html
html/92512.html
html/92513.html
html/92514.html
html/92515.html
html/92516.html
html/92517.html
html/92518.html
html/92519.html
html/9252.html
html/92520.html
html/92521.html
html/92522.html
html/92523.html
html/92524.html
html/92525.html
html/92526.html
html/92527.html
html/92528.html
html/92529.html
html/9253.html
html/92530.html
html/92531.html
html/92532.html
('exception: ', 90796, ' ###')
html/92533.html
html/92534.html
html/92535.html
html/92536.html
html/92537.html
('exception: ', 90801, ' ###')
html/92538.html
html/92539.html
html/9254.html
html/92540.html
html/92541.html
html/92542.html
html/92543.html
html/92544.html
html/92545.html
html/92546.html
html/92547.html
html/92548.html
html/92549.html
html/9255.html
html/92550.html
html/92551.html
html/92552.html
html/92553.html
html/92554.html
html/92555.html
html/92556.html
html/92557.html
html/92558.html

html/9316.html
html/9317.html
('exception: ', 91226, ' ###')
html/9318.html
('exception: ', 91227, ' ###')
html/9319.html
html/932.html
html/9320.html
html/9321.html
html/9322.html
html/9323.html
html/9324.html
html/9325.html
html/9326.html
html/9327.html
html/9328.html
html/9329.html
html/933.html
html/9330.html
html/9331.html
html/9332.html
html/9333.html
html/9334.html
html/9335.html
html/9336.html
html/9337.html
html/9338.html
html/9339.html
html/934.html
html/9340.html
html/9341.html
html/9342.html
html/9343.html
html/9344.html
html/9345.html
html/9346.html
html/9347.html
html/9348.html
html/9349.html
html/935.html
('exception: ', 91262, ' ###')
html/9350.html
html/9351.html
html/9352.html
html/9353.html
html/9354.html
html/9355.html
html/9356.html
html/9357.html
html/9358.html
html/9359.html
html/936.html
html/9360.html
html/9361.html
html/9362.html
html/9363.html
html/9364.html
html/9365.html
html/9366.html
html/9367.html
html/9368.html
html/9369.html
html/937.html
html/9370.htm

html/9743.html
('exception: ', 91697, ' ###')
html/9744.html
html/9745.html
html/9746.html
html/9747.html
html/9748.html
html/9749.html
html/975.html
html/9750.html
html/9751.html
html/9752.html
html/9753.html
html/9754.html
('exception: ', 91709, ' ###')
html/9755.html
html/9756.html
html/9757.html
html/9758.html
('exception: ', 91713, ' ###')
html/9759.html
html/976.html
html/9760.html
html/9761.html
html/9762.html
html/9763.html
html/9764.html
html/9765.html
html/9766.html
html/9767.html
html/9768.html
html/9769.html
html/977.html
html/9770.html
html/9771.html
html/9772.html
html/9773.html
html/9774.html
html/9775.html
html/9776.html
html/9777.html
html/9778.html
html/9779.html
html/978.html
html/9780.html
html/9781.html
html/9782.html
html/9783.html
html/9784.html
html/9785.html
html/9786.html
html/9787.html
('exception: ', 91745, ' ###')
html/9788.html
html/9789.html
html/979.html
('exception: ', 91748, ' ###')
html/9790.html
html/9791.html
('exception: ', 91750, ' ###')
html/9792

In [ ]:
import csv
fieldnames = ['index','fname', 'status']

with open('log_extract.csv', 'wb') as csvfile:
    writer = csv.DictWriter(csvfile,delimiter='\t',quotechar='"',quoting=csv.QUOTE_NONE, fieldnames=fieldnames)
    writer.writer(['test','test','success'])

In [40]:
out_analyst.file.head(1).item()


'1.html'